In [20]:
# === Imports ===
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D, Dense
from tensorflow.keras.optimizers import Adam

# === Load and preprocess German Credit dataset ===
column_names = [
    'Status_checking_account', 'Duration', 'Credit_history', 'Purpose', 'Credit_amount',
    'Savings_account_bonds', 'Employment', 'Installment_rate', 'Personal_status_and_sex',
    'Other_debtors', 'Residence_since', 'Property', 'Age', 'Other_installment_plans',
    'Housing', 'Number_credits', 'Job', 'People_liable', 'Telephone', 'Foreign_worker', 'Risk'
]

df = pd.read_csv("german.data", sep=' ', header=None, names=column_names)
df['target'] = df['Risk'].map({1: 0, 2: 1})
df.drop(columns=['Risk'], inplace=True)

categorical_cols = [
    'Status_checking_account', 'Credit_history', 'Purpose', 'Savings_account_bonds',
    'Employment', 'Personal_status_and_sex', 'Other_debtors', 'Property',
    'Other_installment_plans', 'Housing', 'Job', 'Telephone', 'Foreign_worker'
]
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['target']]

for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

X = df.drop(columns=['target']).values.astype(np.float32)
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_seq = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_seq = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# === Functional BDLSTM Model ===
inputs = Input(shape=(1, X_train.shape[1]))
x = Bidirectional(LSTM(60, return_sequences=True))(inputs)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(80, return_sequences=True))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(120, return_sequences=True))(x)
x = Dropout(0.2)(x)
x = GlobalMaxPooling1D()(x)  # feature representation
outputs = Dense(1, activation='sigmoid')(x)

bdlstm_model = Model(inputs, outputs)
bdlstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# === Train BDLSTM ===
bdlstm_model.fit(X_train_seq, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

# === Extract Features ===
feature_extractor = Model(inputs=bdlstm_model.input, outputs=x)
X_train_feat = feature_extractor.predict(X_train_seq)
X_test_feat = feature_extractor.predict(X_test_seq)

# === Train XGBoost on Extracted Features ===
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_feat, y_train)

# === Evaluate ===
y_pred = xgb.predict(X_test_feat)
y_prob = xgb.predict_proba(X_test_feat)[:, 1]

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📈 F1 Score:", f1_score(y_test, y_pred))
print("🏆 ROC AUC:", roc_auc_score(y_test, y_prob))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.6568 - loss: 0.5995 - val_accuracy: 0.6938 - val_loss: 0.5702
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7605 - loss: 0.5229 - val_accuracy: 0.7125 - val_loss: 0.5706
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7543 - loss: 0.5148 - val_accuracy: 0.7125 - val_loss: 0.5244
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7745 - loss: 0.4939 - val_accuracy: 0.7437 - val_loss: 0.5223
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7709 - loss: 0.4918 - val_accuracy: 0.7250 - val_loss: 0.5207
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8024 - loss: 0.4562 - val_accuracy: 0.7188 - val_loss: 0.5308
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7926 - loss: 0.4646 - val_accuracy: 0.7250 - val_loss: 0.6032
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7583 - loss: 0.4833 - val_accuracy: 0.7250 - v

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:24:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Accuracy: 0.74
📈 F1 Score: 0.5094339622641509
🏆 ROC AUC: 0.7367857142857144

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.86      0.82       140
           1       0.59      0.45      0.51        60

    accuracy                           0.74       200
   macro avg       0.69      0.66      0.67       200
weighted avg       0.73      0.74      0.73       200



In [ ]:
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("📈 F1 Score:", f1_score(y_test, y_pred))
print("🏆 AUC:", roc_auc_score(y_test, y_prob))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step
✅ Accuracy: 0.765
📈 F1 Score: 0.5688073394495413
🏆 ROC AUC: 0.7433333333333334

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.87      0.84       140
           1       0.63      0.52      0.57        60

    accuracy                           0.77       200
   macro avg       0.72      0.69      0.70       200
weighted avg       0.76      0.77      0.76       200

